In [2]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import base64
import os

from animal_shelter_crud import AnimalShelter

# Connect to your MongoDB and read the data into a DataFrame
username = "aacuser"
password = "MyPassword"
shelter = AnimalShelter(username, password)

df = pd.DataFrame(list(shelter.read({})))

# Drop the _id column if it exists
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

app = JupyterDash(__name__)

# Add Grazioso Salvare’s logo
image_filename = os.path.join(os.getcwd(), 'Grazioso Salvare Logo.png')
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Define app layout
app.layout = html.Div([
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image))),
    html.Center(html.B(html.H1('Zachary Maness - SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain Rescue', 'value': 'Mountain'},
                {'label': 'Disaster Rescue', 'value': 'Disaster'},
                {'label': 'Reset', 'value': 'All'}
            ],
            value='All',
            labelStyle={'display': 'block'}
        )
    ]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',
        selected_rows=[],
    ),
    html.Div([
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6'),
    ], className='row'),
])
# These are the breeds associated with each rescue type as per the provided table
breeds_for_rescue = {
    'Water': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'],
    'Mountain': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'],
    'Disaster': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
}

# Callback for interactive data table filter
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_table(filter_type):
    if filter_type == 'All':
        return df.to_dict('records')
    else:
        # Filter based on the breeds associated with each rescue type
        filtered_df = df[df['breed'].isin(breeds_for_rescue.get(filter_type, []))]
        return filtered_df.to_dict('records')

# Callback for geolocation map update
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'selected_rows')]
)
def update_map(viewData, selected_rows):
    if not viewData or not selected_rows:
        return None
    dff = pd.DataFrame.from_dict(viewData)
    row = selected_rows[0]
    latitude = dff.iloc[row, dff.columns.get_loc('location_lat')]
    longitude = dff.iloc[row, dff.columns.get_loc('location_long')]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[latitude, longitude], zoom=10, children=[
            dl.TileLayer(),
            dl.Marker(position=[latitude, longitude], children=[
                dl.Tooltip(dff.iloc[row, dff.columns.get_loc('breed')]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, dff.columns.get_loc('name')])
                ])
            ])
        ])
    ]

# Callback for pie chart update
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_graphs(viewData):
    if not viewData:
        return dcc.Graph(figure=px.pie())
    else:
        dff = pd.DataFrame.from_dict(viewData)
        fig = px.pie(dff, names='breed', title='Distribution of Breeds')
        return dcc.Graph(figure=fig)

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port=8050)



Dash app running on http://127.0.0.1:8050/
